In [1]:
% matplotlib inline

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pylab
import os

pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
# ------------ Looks for larger and darker objects in images -----------------------------------
file_path = os.listdir("Images/")
for infile in file_path:
    if "Ki67" in infile:
        img = cv2.imread('Images/'+infile)
        b,g,r = cv2.split(img)
        rg = cv2.merge([r,g,b])
        rg = rg.astype(np.float32)

        np.seterr(all = 'ignore')
        mask = np.empty_like(rg)
        arr_max = rg.max(-1)
        delta = rg.ptp(-1)
        s = delta/arr_max
        s[delta==0]=0
        idx = (rg[:,:,0] == arr_max)
        mask[idx,0] = np.true_divide((rg[idx,1] - rg[idx,2]), delta[idx])
        idx = (rg[:,:,1] == arr_max)
        mask[idx,0] = 2. + np.true_divide((rg[idx,2] - rg[idx,0]), delta[idx])
        idx = (rg[:,:,2] == arr_max)
        mask[idx,0] = 4. + np.true_divide((rg[idx,0] - rg[idx,1]), delta[idx])
        mask[:,:,0] = ((mask[:,:,0]/6.0)%1.0)*255.0
        mask[:,:,0] = np.nan_to_num(mask[:,:,0])
        mask[:,:,1] = s * 255.0
        mask[:,:,2] = arr_max
        mask = mask.astype(np.uint8)

        points = cv2.inRange(mask, (0, 0, 0), (255,255,165))
        c, _ = cv2.findContours(points, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in c:
            if cv2.contourArea(cnt) > 20000:
                print("{} -- Kicked out".format(infile))
            elif cv2.contourArea(cnt) > 10000:
                print("{} -- Don't count as cell".format(infile))

mp-16-2852_MCT_Ki67_2279.tif -- Kicked out
mp-16-2852_MCT_Ki67_2279.tif -- Don't count as cell
mp-16-2852_MCT_Ki67_2519.tif -- Kicked out
mp-16-2852_MCT_Ki67_2519.tif -- Kicked out
mp-16-2906_MCT_Ki67_693.tif -- Kicked out
mp-16-2906_MCT_Ki67_693.tif -- Kicked out
mp-16-2906_MCT_Ki67_740.tif -- Don't count as cell
mp-16-2906_MCT_Ki67_740.tif -- Kicked out
mp-16-5671_MCT_Ki67_1284.tif -- Kicked out
mp-16-5671_MCT_Ki67_1284.tif -- Don't count as cell
mp-16-5671_MCT_Ki67_1729.tif -- Kicked out
mp-16-5671_MCT_Ki67_2600.tif -- Don't count as cell


In [4]:
print('done')
# HSV 0-255  0-255 0-165
#  - segment the bad area using thresholds above. If contour area > 10000: bad image
#  - if 5000 > contour area > 10000: don't include as general cell

done
